In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 
import datapane as dp
import altair as alt

2022-10-23 11:37:01.950331: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-23 11:37:01.950454: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
plt.rcParams["figure.figsize"] = (16,44)

In [3]:
from settings import OUTPUT_FILE
df = pd.read_csv(OUTPUT_FILE, index_col='model')
df.head()

,P_SUM,U_L1_N,I_SUM,H_TDH_I_L3_N,F,ReacEc_L1,C_phi_L3,ReacEc_L3,RealE_SUM,H_TDH_U_L2_N
model,,,,,,,,,,
LinRegression,0.079265,0.031372,0.065990,0.046060,0.040205,0.000090,0.055232,0.000133,0.000004,0.030794
SimpleLSTM_10_15secs,0.082169,0.057759,0.071507,0.049071,0.040685,0.026302,0.053649,0.064062,0.212368,0.031143
SimpleGRU_10_15secs,0.079773,0.049721,0.071767,0.049153,0.040878,0.012765,0.053661,0.029745,0.232624,0.032727
SimpleLSTM_20_15secs,0.086484,0.052100,0.073630,0.047460,0.040511,0.020637,0.053760,0.052452,0.212594,0.031291
SimpleGRU_20_15secs,0.075620,0.050067,0.070063,0.049064,0.040640,0.007355,0.053538,0.039959,0.217382,0.032436


In [4]:
df.loc[['LinRegression','AutoSKLearn', 'AutoKeras']]

,P_SUM,U_L1_N,I_SUM,H_TDH_I_L3_N,F,ReacEc_L1,C_phi_L3,ReacEc_L3,RealE_SUM,H_TDH_U_L2_N
model,,,,,,,,,,
LinRegression,0.079265,0.031372,0.065990,0.046060,0.040205,0.000090,0.055232,0.000133,0.000004,0.030794
AutoSKLearn,0.063210,0.030960,0.057850,0.034610,0.039560,0.014230,0.044430,0.004760,0.044310,0.030430
AutoKeras,0.144370,0.031636,0.069731,0.050411,0.041842,0.098168,0.052167,0.188699,0.340505,0.032307


In [5]:
control = np.array(df.transpose().LinRegression)
autosklearn = np.array(df.transpose().AutoSKLearn)
autokeras = np.array(df.transpose().AutoKeras)
df = df.drop(['AutoSKLearn', 'AutoKeras', 'LinRegression'])

In [6]:
for i in df.columns:
    print(df[i].nlargest(3))
    print(df[i].nsmallest(3))
    

model
StackedLSTM_5_15secs     0.288565
DropoutLSTM_70_15secs    0.173786
DropoutGRU_70_15secs     0.152263
Name: P_SUM, dtype: float64
model
AttentionBiLSTM_64_20_15secs     0.070771
SimpleLSTM_40_15secs             0.070899
AttentionBiLSTM_128_50_15secs    0.071568
Name: P_SUM, dtype: float64
model
DropoutLSTM_70_15secs    0.071391
DropoutGRU_70_15secs     0.066767
DropoutLSTM_60_15secs    0.066346
Name: U_L1_N, dtype: float64
model
BiGRU_256_10_15secs     0.032347
BiGRU_128_10_15secs     0.032914
BiLSTM_256_10_15secs    0.033583
Name: U_L1_N, dtype: float64
model
BiGRU_256_30_15secs    0.100670
BiGRU_128_30_15secs    0.098324
BiGRU_128_50_15secs    0.096869
Name: I_SUM, dtype: float64
model
AttentionBiLSTM_256_10_15secs    0.065150
AttentionBiLSTM_128_50_15secs    0.065400
DenseGRU_40_15secs               0.067471
Name: I_SUM, dtype: float64
model
DropoutLSTM_70_15secs    0.084035
DropoutGRU_70_15secs     0.082381
DropoutLSTM_60_15secs    0.077738
Name: H_TDH_I_L3_N, dtype: float64


In [ ]:
dfbest = {
    'P_SUM': ['AttentionBiLSTM_64_20_15secs (0.070771)', 'SimpleLSTM_40_15secs (0.070899)', 'AttentionBiLSTM_128_50_15secs (0.071568)'],
    'U_L1_N': ['BiGRU_256_10_15secs (0.032347)', 'BiGRU_128_10_15secs (0.032914)', 'BiGRU2 (0.033223)']
}

dfworst = {
    'P_SUM': ['StackedLSTM_5_15secs (0.288565)', 'DropoutLSTM_70_15secs (0.173786)', 'DropoutGRU_70_15secs (0.152263)'],
    'U_L1_N': ['DropoutLSTM_70_15secs (0.071391)', 'DropoutGRU_70_15secs (0.066767)', 'DropoutLSTM_60_15secs (0.066346)']
}

In [16]:
import plotly.express as px
import plotly.graph_objects as go

for i in range(df.shape[1]):
    df2 = df[df[df.columns[i]] <= control[i]]
    fig = px.scatter(df2, x=df2.index, y=df2.columns[i], height=800, title=df.columns[i] + ' RMSE over different models')
    fig.add_trace(go.Scatter(x=['AutoSKLearn'], y=[autosklearn[i]], mode='markers', marker_symbol = 'star', marker_size = 10))
    fig.add_trace(go.Scatter(x=['AutoKeras'], y=[autokeras[i]], mode='markers', marker_symbol = 'star', marker_size = 10))
    fig.show()

In [ ]:
from settings import CORR_GROUP
plt.rcParams["figure.figsize"] = (4,11)
for k in CORR_GROUP:
    hist = pd.read_csv('results/history_'+k+'.csv')
    fig = px.line(hist, x=hist.columns[1], y=hist.columns[2:], height=800, title=k + ' history')
    fig.show()

In [30]:
import pandas as pd
import tensorflow as tf

import autokeras as ak

from settings import CORR_GROUP
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import netron

def myprint(s):
    with open('modelsummary.txt','a') as f:
        print(s, file=f)

with open('modelsummary.txt','w') as f:
        print("", file=f)

for k in CORR_GROUP:
    netron.start(f'models/{k}_autokeras.h5')
    print(k)
    

Serving 'models/P_SUM_autokeras.h5' at http://localhost:16683
P_SUM
Serving 'models/U_L1_N_autokeras.h5' at http://localhost:24314
U_L1_N
Serving 'models/I_SUM_autokeras.h5' at http://localhost:17670
I_SUM
Serving 'models/H_TDH_I_L3_N_autokeras.h5' at http://localhost:21523
H_TDH_I_L3_N
Serving 'models/F_autokeras.h5' at http://localhost:24203
F
Serving 'models/ReacEc_L1_autokeras.h5' at http://localhost:22165
ReacEc_L1
Serving 'models/C_phi_L3_autokeras.h5' at http://localhost:15983
C_phi_L3
Serving 'models/ReacEc_L3_autokeras.h5' at http://localhost:23468
ReacEc_L3
Serving 'models/RealE_SUM_autokeras.h5' at http://localhost:20693
RealE_SUM
Serving 'models/H_TDH_U_L2_N_autokeras.h5' at http://localhost:18704
H_TDH_U_L2_N


In [31]:
!pip install netron

In [ ]:
message = ""
for k in CORR_GROUP:
    message += """
    \begin{subfigure}[b]{0.5\textwidth}
            \centering
            \includegraphics[width=\textwidth]{img/results/model comparison/"""+k+""" RMSE.png}
            \caption{"""+k+""" RMSE over different models}
            \label{fig:"""+k+""" RMSE graph}
        \end{subfigure}
        \hfill
    """